In [2]:
import numpy as np
import networkx as nx

In [3]:
with open("./data_inputs/day15_input.txt") as f:
    input = f.read()

In [4]:
cave_map = np.array([list(map(int, list(line))) for line in input.split("\n")], dtype=np.uint8)

In [5]:
cave_map

array([[9, 1, 9, ..., 6, 8, 9],
       [9, 4, 5, ..., 2, 3, 9],
       [1, 4, 7, ..., 8, 5, 7],
       ...,
       [8, 2, 1, ..., 8, 4, 9],
       [9, 5, 4, ..., 8, 6, 9],
       [9, 9, 5, ..., 8, 2, 8]], dtype=uint8)

In [6]:
def find_risk(cmap):
    G = nx.DiGraph()
    h, w = cmap.shape
    for x in range(h):
        for y in range(w):
            for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                if 0 <= x+dx < h and 0 <= y+dy < w:
                    G.add_edge((x, y), (x+dx, y+dy), weight=cmap[x+dx, y+dy])

    dist, _ = nx.single_source_dijkstra(G, (0, 0), target=(h-1, w-1))
    return dist

# ----- PART 1 -----
print("Result1:", find_risk(cave_map))

# ----- PART 2 -----
large_map = np.zeros((500, 500))

for x in range(5):
    for y in range(5):
        large_map[x*100:(x+1)*100, y*100:(y+1)*100] = cave_map + (x+y)

large_map[large_map > 9] -= 9

print("Result2:", find_risk(large_map))

Result1: 741
Result2: 2976.0


In [ ]:
# Alternative try where I tried to implement Dijikstra from scratch but took too long and failed:

In [ ]:
from queue import PriorityQueue

In [ ]:
class Graph:
    def __init__(self, vertices):
        self.v = vertices
        self.edges = {}
        self.visited = []
        self.neighbors = {v: [] for v in vertices}

    def add_edge(self, u, v, weight):
        self.edges[(u, v)] = weight
        self.edges[(v, u)] = weight
        self.neighbors[u].append(v)

In [ ]:
@lru_cache
def dijikstra(graph, start_vertex):
    D = {v:float('inf') for v in graph.v}
    D[start_vertex] = 0

    pq = PriorityQueue()
    pq.put((0, start_vertex))

    while not pq.empty():
        (dist, current_vertex) = pq.get()
        graph.visited.append(current_vertex)

        for neighbor in graph.neighbors[current_vertex]:
            if graph.edges[(current_vertex, neighbor)] != -1:
                distance = graph.edges[(current_vertex, neighbor)]
                if neighbor not in graph.visited:
                    old_cost = D[neighbor]
                    new_cost = D[current_vertex] + distance
                    if new_cost < old_cost:
                        pq.put((new_cost, neighbor))
                        D[neighbor] = new_cost

    return D

In [ ]:
large_map = np.zeros((500, 500))

for i in range(5):
    for j in range(5):
        large_map[i*100:(i+1)*100, j*100:(j+1)*100] = cave_map_ini + (i+j)

large_map[large_map > 9] -= 9

In [ ]:
g = Graph([(x, y) for x in range(500) for y in range(500)])

In [ ]:
for x in range(large_map.shape[0]):
    for y in range(large_map.shape[1]):
        for dy, dx in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
            if 0 <= x+dx < large_map.shape[0] and 0 <= y+dy < large_map.shape[1]:
                g.add_edge((x, y), (x+dx, y+dy), large_map[x+dx, y+dy])
                

In [ ]:
large_map

array([[9., 1., 9., ..., 1., 3., 4.],
       [9., 4., 5., ..., 6., 7., 4.],
       [1., 4., 7., ..., 3., 9., 2.],
       ...,
       [3., 6., 5., ..., 7., 3., 8.],
       [4., 9., 8., ..., 7., 5., 8.],
       [4., 4., 9., ..., 7., 1., 7.]])

In [ ]:
D[498, 499]

2960.0